In [11]:
import pandas as pd
import numpy as np
import os
import scipy.linalg as linalg
from pulp import *
import csv
import itertools
from tqdm import tqdm
os.chdir('/home/valesco/Datasets/NFL_projs')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [39]:
def home_away(x):
    if '@' in x:
        return 'away'
    else:
        return 'home'

proj_df = pd.read_csv('week3_dk_proj.csv')
cov_df = pd.read_csv('nfl_covariance_matrix.csv', index_col = 0)

#number of games in spreadsheet plus one
num_of_games = 17


proj_df['matchup'] = tuple(zip(proj_df['team'], proj_df['opponent']))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: x.replace('$',''))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: float(x.replace('K', '')))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: x * 1000)
proj_df['rank'] = proj_df.groupby(['team', 'position'])['points_avg'].rank(ascending = False, method = 'max')

proj_df['rank'].fillna(value = 9999, inplace = True)
proj_df['salary'].fillna(value = 9999, inplace = True)
proj_df['points_avg'].fillna(value = -9.999, inplace = True)
proj_df['rank'] = proj_df['rank'].apply(lambda x: str(int(x)))
proj_df['home_away'] = proj_df['opponent'].apply(lambda x: home_away(x))
proj_df['rank'] = proj_df['home_away'] + '_' + proj_df['position'] + proj_df['rank']

proj_df.to_csv('2016_week3_proj_process.csv')
proj_df.head()

,player_id,name,salary,team,opponent,points_avg,points_floor,floor_pct,floor_pt_per_dpct,points_ceiling,ceiling_pct,consistency,position,position_count,position_id,positions,adj_ypa,event_team_id,expected_sack_pct,Properties.ImpPts,Properties.InterceptionPct,Properties.IsProjected,Properties.MktShrPct,Properties.Month_FPOpp,Properties.Month_FPSnap,...,Properties.SackPct,Properties.Salary,Properties.Salary_Movement,Properties.Score,Properties.Season_FPOpp,Properties.Season_FPSnap,Properties.Season_PPG,Properties.Season_PPG_Percentile,Properties.Season_Plus_Minus,Properties.Season_Salary_Change,Properties.Season_X0,Properties.Season_X1,Properties.Season_X2,Properties.Site_Salary,Properties.SourceId,Properties.TakeawayPct,Properties.Team,Properties.TouchdownAllowedPct,Properties.Upside,Properties.Vegas,Properties.YardsPerPassingAttempt,Score,matchup,rank,home_away
0,926,Davante Adams,4100,GB,DET,10.1,6.0,62,62,14.2,61,50,WR,1,103,['WR'],NaN,32780402,14,8.50,2.70,True,58.0,0.0,0.17,...,8.29,4100,79,40.0,0.00,0.14,8.2,56,-0.5,-200,38.0,38.0,23.0,64,4,1.56,GB,3.63,40,90,7.34,40.0,"(GB, DET)",home_WR3,home
1,969,Kamar Aiken,3100,BAL,@JAX,5.8,3.0,38,38,8.6,39,58,WR,1,103,['WR'],NaN,32780501,53,6.21,1.17,True,82.0,0.0,0.05,...,7.20,3100,99,53.0,0.00,0.21,11.9,75,2.7,-1600,25.0,43.0,31.0,99,4,0.65,BAL,4.28,53,74,7.42,53.0,"(BAL, @JAX)",away_WR4,away
2,1007,Dwayne Allen,3800,IND,SD,10.1,6.8,83,83,13.4,81,45,TE,1,104,['TE'],NaN,32781202,29,6.85,2.81,True,36.0,0.0,0.20,...,5.96,3800,7,37.0,0.00,0.08,3.4,42,-1.4,600,61.0,30.0,7.0,92,4,1.63,IND,4.68,18,86,7.50,37.0,"(IND, SD)",home_TE1,home
3,1047,Danny Amendola,3700,NE,HOU,4.1,2.2,32,32,6.0,30,48,WR,1,103,['WR'],36.0,32748802,32,7.58,1.64,True,68.0,0.0,0.73,...,6.72,3700,88,87.0,1.66,0.33,12.5,77,3.8,-500,14.0,35.0,50.0,97,4,1.02,NE,3.13,87,34,7.01,87.0,"(NE, HOU)",home_WR3,home
4,1055,C.J. Anderson,7300,DEN,@CIN,17.6,11.9,93,93,23.3,94,41,RB,1,102,['RB'],NaN,32780301,12,16.98,2.04,True,82.0,0.0,0.51,...,9.00,7300,4,84.0,0.00,0.37,12.3,82,3.0,1400,33.0,26.0,40.0,2,4,1.21,DEN,3.50,68,24,6.16,84.0,"(DEN, @CIN)",away_RB1,away


In [40]:
cov_df.head(15)

,home_D1,home_QB1,home_RB1,home_RB2,home_TE1,home_WR1,home_WR2,home_WR3,away_D1,away_QB1,away_RB1,away_RB2,away_TE1,away_WR1,away_WR2,away_WR3
home_D1,1.00,-0.08,0.11,0.03,-0.08,0.01,0.00,-0.09,-0.24,-0.41,-0.25,-0.13,-0.09,-0.18,-0.12,-0.05
home_QB1,-0.08,1.00,0.08,0.06,0.33,0.46,0.33,0.28,-0.41,0.20,0.03,-0.01,0.09,0.09,0.07,0.13
home_RB1,0.11,0.08,1.00,-0.13,-0.06,0.03,-0.03,-0.07,-0.25,0.03,-0.01,-0.06,0.03,0.00,0.01,0.03
home_RB2,0.03,0.06,-0.13,1.00,0.04,-0.03,-0.07,0.00,-0.13,-0.01,-0.06,-0.05,0.00,-0.02,-0.03,0.00
home_TE1,-0.08,0.33,-0.06,0.04,1.00,-0.04,-0.06,0.00,-0.09,0.09,0.03,0.00,0.04,0.02,0.04,0.03
home_WR1,0.01,0.46,0.03,-0.03,-0.04,1.00,0.04,0.00,-0.18,0.09,0.00,-0.02,0.02,0.06,0.02,0.06
home_WR2,0.00,0.33,-0.03,-0.07,-0.06,0.04,1.00,0.01,-0.12,0.07,0.01,-0.03,0.04,0.02,0.00,0.04
home_WR3,-0.09,0.28,-0.07,0.00,0.00,0.00,0.01,1.00,-0.05,0.13,0.03,0.00,0.03,0.06,0.04,0.10
away_D1,-0.24,-0.41,-0.25,-0.13,-0.09,-0.18,-0.12,-0.05,1.00,-0.08,0.11,0.03,-0.08,0.01,0.00,-0.09
away_QB1,-0.41,0.20,0.03,-0.01,0.09,0.09,0.07,0.13,-0.08,1.00,0.08,0.06,0.33,0.46,0.33,0.28


In [41]:
away_teams = proj_df['team'].loc[proj_df['home_away'] == 'away'].unique()

player_cols = []
count = 0

for team in away_teams:
    temp1 = proj_df.loc[proj_df['team'] == team]
    temp2 = proj_df.loc[proj_df['opponent'] == team]
    temp_df = pd.concat([temp1, temp2], axis = 0)
    
    cols = cov_df.columns

    player_ls = []
    for c in cols:
        player = pd.DataFrame(temp_df.loc[temp_df['rank'] == c])
        
        if len(player) < 1:
            name = 'Not Available_' + str(count)
            count += 1
        else:
            name = player.iloc[0]['name']
        player_ls.append(name)
        
    player_cols.extend(player_ls)
    

arr_cov = np.asarray(cov_df)
for i in range(1, num_of_games):
    arr_new = np.asarray(cov_df)
    arr_cov = linalg.block_diag(arr_cov, arr_new)
    
comp_cov_df = pd.DataFrame(data = arr_cov, index = player_cols, columns = player_cols)
comp_cov_df.head()

,Jacksonville Defense,Blake Bortles,T.J. Yeldon,Chris Ivory,Julius Thomas,Allen Robinson,Allen Hurns,Marqise Lee,Baltimore Defense,Joe Flacco,Justin Forsett,Terrance West,Dennis Pitta,Mike Wallace,Steve Smith,Breshad Perriman,Not Available_0,Not Available_1,Not Available_2,Giovani Bernard,Not Available_3,Not Available_4,A.J. Green,Not Available_5,Denver Defense,...,Danny Amendola,Houston Defense,Brock Osweiler,Lamar Miller,Tyler Ervin,Stephen Anderson,DeAndre Hopkins,Will Fuller,Jaelen Strong,Dallas Defense,Dak Prescott,Ezekiel Elliott,Not Available_16,Jason Witten,Dez Bryant,Cole Beasley,Terrance Williams,Chicago Defense,Brian Hoyer,Jeremy Langford,Jordan Howard,Zach Miller,Alshon Jeffery,Kevin White,Eddie Royal
Jacksonville Defense,1.00,-0.08,0.11,0.03,-0.08,0.01,0.00,-0.09,-0.24,-0.41,-0.25,-0.13,-0.09,-0.18,-0.12,-0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blake Bortles,-0.08,1.00,0.08,0.06,0.33,0.46,0.33,0.28,-0.41,0.20,0.03,-0.01,0.09,0.09,0.07,0.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T.J. Yeldon,0.11,0.08,1.00,-0.13,-0.06,0.03,-0.03,-0.07,-0.25,0.03,-0.01,-0.06,0.03,0.00,0.01,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chris Ivory,0.03,0.06,-0.13,1.00,0.04,-0.03,-0.07,0.00,-0.13,-0.01,-0.06,-0.05,0.00,-0.02,-0.03,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Julius Thomas,-0.08,0.33,-0.06,0.04,1.00,-0.04,-0.06,0.00,-0.09,0.09,0.03,0.00,0.04,0.02,0.04,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
in_file = "2016_week3_proj_process.csv"
reader = csv.DictReader(open(in_file, 'r'))
player_dict = []
for i , row in enumerate(reader):
    team = {k: v for k, v in row.items()}
    team['id'] = i
    player_dict.append(team)
    
players = [player['id'] for player in player_dict]
teams = [0]

In [54]:
num_teams = 100
row_count = 0
opt_df = []
opt_df = pd.DataFrame(columns = ['qb_name', 'qb_pos', 'qb_points', 'qb_salary',
                                 'wr1_name', 'wr1_pos', 'wr1_points', 'wr1_salary',
                                 'wr2_name', 'wr2_pos', 'wr2_points', 'wr2_salary',
                                 'wr3_name', 'wr3_pos', 'wr3_points', 'wr3_salary',
                                 'rb1_name', 'rb1_pos', 'rb1_points', 'rb1_salary',
                                 'rb2_name', 'rb2_pos', 'rb2_points', 'rb2_salary',
                                 'te_name', 'te_pos', 'te_points', 'te_salary',
                                 'flex_name', 'flex_pos', 'flex_points', 'flex_salary',
                                 'def_name', 'def_pos', 'def_points', 'def_salary',
                                 'team_points', 'total_salary'
                                ])

#time_ls = ['week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8', 'week9',
#           'week10', 'week11', 'week12', 'week13', 'week14', 'week15', 'week16', 'week17']

#player_dict = []
#for player in player_dict_pre:
#    if player['season_year'] == '2015' and player['week'] == '17':
#        player_dict.append(player)

master_ls = {}
        
#for time in tqdm(time_ls):
for i, t1 in enumerate(tqdm(range(num_teams))):
    team_dict = {}

    if i == 0:
        # Set objective
        players = [player['id'] for player in player_dict]            

        opt = LpProblem('Optimizer', LpMaximize)
        possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

        #objective
        opt += sum([float(player['points_avg']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])

        #set constraints
        for team in teams:
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 9, '9 players per team '+str(team)
            opt += lpSum([float(player['salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'QB']) == 1, 'one QB per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'WR']) >= 3, 'three or more WR per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'RB']) >= 2, 'two or more RB per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'TE']) >= 1, 'two or more TE per team '
            opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'D']) == 1, 'two or more DEF per team '
            #opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['time_line'] == time]) == 9, 'same week'

        opt.writeLP('fantasy_optimization.lp')
        opt.solve()
        status = LpStatus[opt.status]
        selected = []
        
        points = 0

        for player in player_dict:
            if possible_assignments[0][player['id']].value() == 1.0:
                players = {}
                #print(player['player'] + ' salary: ' + player['salary'] + ' proj: ' + player['odds'])
                #opt_odds.append(float(player['odds']))
                name = player['name']
                players['name'] = player['name']
                players['position'] = player['position']
                players['points_avg'] = player['points_avg']
                #players['year'] = '2015'
                #players['week'] = player['week']
                players['salary'] = player['salary']
                #players['time_line'] = player['time_line']
                selected.append(player['id'])
                team_dict[player['id']] = players
                
                points += float(players['points_avg'])

        master_ls[i] = team_dict
        points_last = points
    else:
        # Set objective
        players = [player['id'] for player in player_dict]

        opt = LpProblem('Optimizer', LpMaximize)
        possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

        #objective
        opt += sum([float(player['points_avg']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])
        
        #Set no equal teams constraint
        if points < points_last:
            selected = selected[-9:]
            
        points_last = points
        
        c = 0
        for s in range(int(len(selected)/9)):
            opt += lpSum(possible_assignments[0][selected[c]] + possible_assignments[0][selected[c+1]] +
                          possible_assignments[0][selected[c+2]] + possible_assignments[0][selected[c+3]] +
                          possible_assignments[0][selected[c+4]] + possible_assignments[0][selected[c+5]] +
                          possible_assignments[0][selected[c+6]] + possible_assignments[0][selected[c+7]] +
                          possible_assignments[0][selected[c+8]]) <= 8, 'selected' + str(s)
            
            
            c += 9

        #set constraints

        opt += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 9, '6 players per team '+str(team)
        opt += lpSum([float(player['salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
        opt += lpSum([float(player['points_avg']) * possible_assignments[team][player['id']] for player in player_dict]) <= points_last
        opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'QB']) == 1, 'one QB per team '
        opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'WR']) >= 3, 'three or more WR per team '
        opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'RB']) >= 2, 'two or more RB per team '
        opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'TE']) >= 1, 'two or more TE per team '
        opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['position'] == 'D']) == 1, 'two or more DEF per team '
        #opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['time_line'] == time]) == 9, 'same week'

        opt.writeLP('fantasy_optimization.lp')
        opt.solve()
        status = LpStatus[opt.status]
        
        total_points = 0

        for player in player_dict:
            if possible_assignments[0][player['id']].value() == 1.0:
                players = {}
                #print(player['player'] + ' salary: ' + player['salary'] + ' proj: ' + player['odds'])
                #opt_odds.append(float(player['odds']))
                name = player['name']
                players['name'] = player['name']
                players['position'] = player['position']
                players['points_avg'] = player['points_avg']
                #players['season_year'] = '2015'
                #players['week'] = player['week']
                players['salary'] = player['salary']
                #players['time_line'] = player['time_line']
                selected.append(player['id'])
                team_dict[player['id']] = players
                
        

        master_ls[i] = team_dict
    
    points = 0
    salary = 0
    wr_count = 0
    rb_count = 0
    te_count = 0  
   
    for k, v in team_dict.items():
        

        points += float(v['points_avg'])
        salary += float(v['salary'])
        if v['position'] == 'QB':
            qb_name = v['name']
            qb_pos = 'QB'
            qb_points = v['points_avg']
            qb_salary = v['salary']
            #year = v1['season_year']
            #year = '2015'
            #week = v1['week']
            #timeline = v1['time_line']
        elif v['position'] == 'WR':
            if wr_count == 0:
                wr1_name = v['name']
                wr1_pos = 'WR1'
                wr1_points = v['points_avg']
                wr1_salary = v['salary']
            elif wr_count == 1:
                wr2_name = v['name']
                wr2_pos = 'WR2'
                wr2_points = v['points_avg']
                wr2_salary = v['salary']
            elif wr_count == 2:
                wr3_name = v['name']
                wr3_pos = 'WR3'
                wr3_points = v['points_avg']
                wr3_salary = v['salary']
            else:
                flex_name = v['name']
                flex_pos = 'flex_WR'
                flex_points = v['points_avg']
                flex_salary = v['salary']
            wr_count += 1
        elif v['position'] == 'RB':
            if rb_count == 0:
                rb1_name = v['name']
                rb1_pos = 'RB1'
                rb1_points = v['points_avg']
                rb1_salary = v['salary']
            elif rb_count == 1:
                rb2_name = v['name']
                rb2_pos = 'RB2'
                rb2_points = v['points_avg']
                rb2_salary = v['salary']
            else:
                flex_name = v['name']
                flex_pos = 'flex_RB'
                flex_points = v['points_avg']
                flex_salary = v['salary']
            rb_count += 1
        elif v['position'] == 'TE':
            if te_count == 0:
                te_name = v['name']
                te_pos = 'TE'
                te_points = v['points_avg']
                te_salary = v['salary']
            else:
                flex_name = v['name']
                flex_pos = 'flex_TE'
                flex_points = v['points_avg']
                flex_salary = v['salary']
            te_count += 1
        elif v['position'] == 'D':
            def_name = v['name']
            def_pos = 'D'
            def_points = v['points_avg']
            def_salary = v['salary']
        else:
            print('ERROR!!!')


    opt_df.loc[row_count] = [qb_name, qb_pos, qb_points, qb_salary, wr1_name, wr1_pos, wr1_points, wr1_salary,
                             wr2_name, wr2_pos, wr2_points, wr2_salary, wr3_name, wr3_pos, wr3_points, wr3_salary,
                             rb1_name, rb1_pos, rb1_points, rb1_salary, rb2_name, rb2_pos, rb2_points, rb2_salary,
                            te_name, te_pos, te_points, te_salary, flex_name, flex_pos, flex_points, flex_salary,
                             def_name, def_pos, def_points, def_salary, points, salary]

    row_count += 1

100%|██████████| 100/100 [00:25<00:00,  3.70it/s]


In [55]:
opt_df.head()

,qb_name,qb_pos,qb_points,qb_salary,wr1_name,wr1_pos,wr1_points,wr1_salary,wr2_name,wr2_pos,wr2_points,wr2_salary,wr3_name,wr3_pos,wr3_points,wr3_salary,rb1_name,rb1_pos,rb1_points,rb1_salary,rb2_name,rb2_pos,rb2_points,rb2_salary,te_name,te_pos,te_points,te_salary,flex_name,flex_pos,flex_points,flex_salary,def_name,def_pos,def_points,def_salary,team_points,total_salary
0,Andy Dalton,QB,21.8,6100,Stefon Diggs,WR1,16.3,5100,Antonio Brown,WR2,25.1,9600,Travis Benjamin,WR3,17.7,5200,Melvin Gordon,RB1,19.1,5800,Charles Sims,RB2,16.4,4900,Dennis Pitta,TE,10.9,3400,DeAngelo Williams,flex_RB,22.5,7500,Washington Defense,D,8.0,2400,157.8,50000.0
1,Andy Dalton,QB,21.8,6100,Stefon Diggs,WR1,16.3,5100,Julio Jones,WR2,24.9,9500,Travis Benjamin,WR3,17.7,5200,Charles Sims,RB1,16.4,4900,Melvin Gordon,RB2,19.1,5800,Dennis Pitta,TE,10.9,3400,DeAngelo Williams,flex_RB,22.5,7500,Washington Defense,D,8.0,2400,157.6,49900.0
2,Andy Dalton,QB,21.8,6100,Jarvis Landry,WR1,19.1,6600,Terrelle Pryor,WR2,13.3,3400,Antonio Brown,WR3,25.1,9600,Melvin Gordon,RB1,19.1,5800,DeAngelo Williams,RB2,22.5,7500,Dennis Pitta,TE,10.9,3400,Travis Benjamin,flex_WR,17.7,5200,Washington Defense,D,8.0,2400,157.5,50000.0
3,Andy Dalton,QB,21.8,6100,Antonio Brown,WR1,25.1,9600,Travis Benjamin,WR2,17.7,5200,Stefon Diggs,WR3,16.3,5100,Melvin Gordon,RB1,19.1,5800,Charles Sims,RB2,16.4,4900,Jacob Tamme,TE,10.5,3200,DeAngelo Williams,flex_RB,22.5,7500,Washington Defense,D,8.0,2400,157.4,49800.0
4,Andy Dalton,QB,21.8,6100,Antonio Brown,WR1,25.1,9600,Travis Benjamin,WR2,17.7,5200,Stefon Diggs,WR3,16.3,5100,Melvin Gordon,RB1,19.1,5800,Charles Sims,RB2,16.4,4900,Jacob Tamme,TE,10.5,3200,DeAngelo Williams,flex_RB,22.5,7500,Oakland Defense,D,8.0,2600,157.4,50000.0


In [361]:
opt_df.to_csv('1k_optimal_lineups_wk1.csv')
opt_df.to_pickle('1k_optimal_lineups_wk1.p')

In [58]:
def sort_df(df):
    sorted_df = pd.DataFrame(columns = ['qb_name', 'qb_pos', 'qb_points', 'qb_salary',
                                     'wr1_name', 'wr1_pos', 'wr1_points', 'wr1_salary',
                                     'wr2_name', 'wr2_pos', 'wr2_points', 'wr2_salary',
                                     'wr3_name', 'wr3_pos', 'wr3_points', 'wr3_salary',
                                     'rb1_name', 'rb1_pos', 'rb1_points', 'rb1_salary',
                                     'rb2_name', 'rb2_pos', 'rb2_points', 'rb2_salary',
                                     'te_name', 'te_pos', 'te_points', 'te_salary',
                                     'flex_name', 'flex_pos', 'flex_points', 'flex_salary',
                                     'def_name', 'def_pos', 'def_points', 'def_salary',
                                     'team_points', 'total_salary'
                                    ])

    for index, row in df.iterrows():
        qb_name = row['qb_name']
        qb_points = row['qb_points']
        qb_salary = row['qb_salary']

        def_name = row['def_name']
        def_points = row['def_points']
        def_salary = row['def_salary']


        team_points = row['team_points']
        total_salary = row['total_salary']

        wrA = [row['wr1_name'], row['wr1_points'], row['wr1_salary']]
        wrB = [row['wr2_name'], row['wr2_points'], row['wr2_salary']]
        wrC = [row['wr3_name'], row['wr3_points'], row['wr3_salary']]

        rbA = [row['rb1_name'], row['rb1_points'], row['rb1_salary']]
        rbB = [row['rb2_name'], row['rb2_points'], row['rb2_salary']]

        teA = [row['te_name'], row['te_points'], row['te_salary']]


        if row['flex_pos'] == 'flex_WR':
            wrD = [row['flex_name'], row['flex_points'], row['flex_salary']]
            wr_ls = [wrA, wrB, wrC, wrD]
        else:
            wr_ls = [wrA, wrB, wrC]

        if row['flex_pos'] == 'flex_RB':
            rbC = [row['flex_name'], row['flex_points'], row['flex_salary']]
            rb_ls = [rbA, rbB, rbC]
        else:
            rb_ls = [rbA, rbB]

        if row['flex_pos'] == 'flex_TE':
            teB = [row['flex_name'], row['flex_points'], row['flex_salary']]
            te_ls = [teA, teB]
        else:
            te_ls = [teA]  


        wr_ls.sort(key = lambda x: x[2], reverse = True)
        rb_ls.sort(key = lambda x: x[2], reverse = True)
        te_ls.sort(key = lambda x: x[2], reverse = True)

        for i, wr in enumerate(wr_ls):
            if i == 0:
                wr1_name = wr[0]
                wr1_points = wr[1]
                wr1_salary = wr[2]
            elif i == 1:
                wr2_name = wr[0]
                wr2_points = wr[1]
                wr2_salary = wr[2]
            elif i == 2:
                wr3_name = wr[0]
                wr3_points = wr[1]
                wr3_salary = wr[2]
            else:
                flex_name = wr[0]
                flex_points = wr[1]
                flex_salary = wr[2]
                flex_pos = 'flex_WR'

        for i, rb in enumerate(rb_ls):
            if i == 0:
                rb1_name = rb[0]
                rb1_points = rb[1]
                rb1_salary = rb[2]
            elif i == 1:
                rb2_name = rb[0]
                rb2_points = rb[1]
                rb2_salary = rb[2]
            else:
                flex_name = rb[0]
                flex_points = rb[1]
                flex_salary = rb[2]
                flex_pos = 'flex_RB'
        for i, te in enumerate(te_ls):
            if i == 0:
                te_name = te[0]
                te_points = te[1]
                te_salary = te[2]
            else:
                flex_name = te[0]
                flex_points = te[1]
                flex_salary = te[2]
                flex_pos = 'flex_TE'



        sorted_df.loc[index] = [qb_name, 'QB', qb_points, qb_salary,
                             wr1_name, 'WR1', wr1_points, wr1_salary,
                             wr2_name, 'WR2', wr2_points, wr2_salary,
                             wr3_name, 'WR3', wr3_points, wr3_salary,
                             rb1_name, 'RB1', rb1_points, rb1_salary,
                             rb2_name, 'RB2', rb2_points, rb2_salary,
                             te_name, 'TE', te_points, te_salary,
                             flex_name, flex_pos, flex_points, flex_salary,
                             def_name, 'DEF', def_points, def_salary,
                             team_points, total_salary]




    return sorted_df

sorted_df = sort_df(opt_df)
sorted_df.head()

,qb_name,qb_pos,qb_points,qb_salary,wr1_name,wr1_pos,wr1_points,wr1_salary,wr2_name,wr2_pos,wr2_points,wr2_salary,wr3_name,wr3_pos,wr3_points,wr3_salary,rb1_name,rb1_pos,rb1_points,rb1_salary,rb2_name,rb2_pos,rb2_points,rb2_salary,te_name,te_pos,te_points,te_salary,flex_name,flex_pos,flex_points,flex_salary,def_name,def_pos,def_points,def_salary,team_points,total_salary
0,Andy Dalton,QB,21.8,6100,Antonio Brown,WR1,25.1,9600,Travis Benjamin,WR2,17.7,5200,Stefon Diggs,WR3,16.3,5100,DeAngelo Williams,RB1,22.5,7500,Melvin Gordon,RB2,19.1,5800,Dennis Pitta,TE,10.9,3400,Charles Sims,flex_RB,16.4,4900,Washington Defense,DEF,8.0,2400,157.8,50000.0
1,Andy Dalton,QB,21.8,6100,Julio Jones,WR1,24.9,9500,Travis Benjamin,WR2,17.7,5200,Stefon Diggs,WR3,16.3,5100,DeAngelo Williams,RB1,22.5,7500,Melvin Gordon,RB2,19.1,5800,Dennis Pitta,TE,10.9,3400,Charles Sims,flex_RB,16.4,4900,Washington Defense,DEF,8.0,2400,157.6,49900.0
2,Andy Dalton,QB,21.8,6100,Antonio Brown,WR1,25.1,9600,Jarvis Landry,WR2,19.1,6600,Travis Benjamin,WR3,17.7,5200,DeAngelo Williams,RB1,22.5,7500,Melvin Gordon,RB2,19.1,5800,Dennis Pitta,TE,10.9,3400,Terrelle Pryor,flex_WR,13.3,3400,Washington Defense,DEF,8.0,2400,157.5,50000.0
3,Andy Dalton,QB,21.8,6100,Antonio Brown,WR1,25.1,9600,Travis Benjamin,WR2,17.7,5200,Stefon Diggs,WR3,16.3,5100,DeAngelo Williams,RB1,22.5,7500,Melvin Gordon,RB2,19.1,5800,Jacob Tamme,TE,10.5,3200,Charles Sims,flex_RB,16.4,4900,Washington Defense,DEF,8.0,2400,157.4,49800.0
4,Andy Dalton,QB,21.8,6100,Antonio Brown,WR1,25.1,9600,Travis Benjamin,WR2,17.7,5200,Stefon Diggs,WR3,16.3,5100,DeAngelo Williams,RB1,22.5,7500,Melvin Gordon,RB2,19.1,5800,Jacob Tamme,TE,10.5,3200,Charles Sims,flex_RB,16.4,4900,Oakland Defense,DEF,8.0,2600,157.4,50000.0


In [62]:
team_ls

['Andy Dalton',
 'Antonio Brown',
 'Jarvis Landry',
 'Travis Benjamin',
 'DeAngelo Williams',
 'Melvin Gordon',
 'Dennis Pitta',
 'Washington Defense',
 'Terrelle Pryor']

In [60]:
def add_corr_col(df, cov_df):
    corr_col = pd.DataFrame(columns = ['corr_sum'])
    for index, row in df.iterrows():
        corr_sum = 0

        qb = row['qb_name']
        wr1 = row['wr1_name']
        wr2 = row['wr2_name']
        wr3 = row['wr3_name']
        rb1 = row['rb1_name']
        rb2 = row['rb2_name']
        te = row['te_name']
        d = row['def_name']
        flex = row['flex_name']

        team_ls = [qb, wr1, wr2, wr3, rb1, rb2, te, d, flex]


        for tup in itertools.combinations(team_ls, 2):
            try:
                covar = cov_df.loc[tup[0]][tup[1]]
            except:
                covar = 0
            corr_sum += covar

        corr_col.loc[index] = corr_sum


    return corr_col
    
column = add_corr_col(sorted_df, comp_cov_df)


ValueError: cannot reindex from a duplicate axis

In [395]:
opt_df2 = opt_df.join(column)
opt_df2.sort_values('corr_sum', ascending = False, inplace = True)
opt_df2.reset_index(inplace = True)

In [396]:
opt_df2.head(15)

,index,qb_name,qb_pos,qb_points,qb_salary,wr1_name,wr1_pos,wr1_points,wr1_salary,wr2_name,wr2_pos,wr2_points,wr2_salary,wr3_name,wr3_pos,wr3_points,wr3_salary,rb1_name,rb1_pos,rb1_points,rb1_salary,rb2_name,rb2_pos,rb2_points,rb2_salary,te_name,te_pos,te_points,te_salary,flex_name,flex_pos,flex_points,flex_salary,def_name,def_pos,def_points,def_salary,team_points,total_salary,corr_sum
0,33,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Austin Seferian-Jenkins,TE,10.19,2800,Greg Olsen,flex_TE,16.32,5100,Detroit Lions,DST,8.23,2500,157.90,49800,0.47
1,9,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Austin Seferian-Jenkins,TE,10.19,2800,Greg Olsen,flex_TE,16.32,5100,Indianapolis Colts,DST,8.65,2700,158.32,50000,0.47
2,84,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,Lamar Miller,RB2,18.88,7000,Austin Seferian-Jenkins,TE,10.19,2800,Greg Olsen,flex_TE,16.32,5100,Indianapolis Colts,DST,8.65,2700,157.54,50000,0.44
3,37,Dak Prescott,QB,18.73,5000,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Jason Witten,TE,12.15,4100,Greg Olsen,flex_TE,16.32,5100,Green Bay Packers,DST,9.08,2800,157.86,49800,0.36
4,24,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Greg Olsen,TE,16.32,5100,Crockett Gillmore,flex_TE,10.03,2600,Pittsburgh Steelers,DST,8.58,2900,158.09,50000,0.24
5,36,Dak Prescott,QB,18.73,5000,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Zach Ertz,TE,14.12,4300,Coby Fleener,flex_TE,14.87,4900,Pittsburgh Steelers,DST,8.58,2900,157.88,49900,0.15
6,81,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Austin Seferian-Jenkins,TE,10.19,2800,Greg Olsen,flex_TE,16.32,5100,Tampa Bay Buccaneers,DST,7.88,2600,157.55,49900,0.13
7,49,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Greg Olsen,TE,16.32,5100,Crockett Gillmore,flex_TE,10.03,2600,Detroit Lions,DST,8.23,2500,157.74,49600,0.12
8,83,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Michael Crabtree,WR2,16.12,5500,Julio Jones,WR3,27.57,9400,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Crockett Gillmore,TE,10.03,2600,Tajae Sharpe,flex_WR,11.83,3000,Detroit Lions,DST,8.23,2500,157.54,50000,0.12
9,18,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Julio Jones,WR2,27.57,9400,Tajae Sharpe,WR3,11.83,3000,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Greg Olsen,TE,16.32,5100,Crockett Gillmore,flex_TE,10.03,2600,Indianapolis Colts,DST,8.65,2700,158.16,49800,0.12


In [403]:
opt_df2.loc[opt_df2['wr3_name'] == 'Julio Jones']

,index,qb_name,qb_pos,qb_points,qb_salary,wr1_name,wr1_pos,wr1_points,wr1_salary,wr2_name,wr2_pos,wr2_points,wr2_salary,wr3_name,wr3_pos,wr3_points,wr3_salary,rb1_name,rb1_pos,rb1_points,rb1_salary,rb2_name,rb2_pos,rb2_points,rb2_salary,te_name,te_pos,te_points,te_salary,flex_name,flex_pos,flex_points,flex_salary,def_name,def_pos,def_points,def_salary,team_points,total_salary,corr_sum
8,83,Jameis Winston,QB,21.58,6600,Antonio Brown,WR1,30.3,9800,Michael Crabtree,WR2,16.12,5500,Julio Jones,WR3,27.57,9400,James White,RB1,12.22,3600,DeAngelo Williams,RB2,19.66,7000,Crockett Gillmore,TE,10.03,2600,Tajae Sharpe,flex_WR,11.83,3000,Detroit Lions,DST,8.23,2500,157.54,50000,0.12


In [405]:
comp_cov_df.loc['Jameis Winston']['Julio Jones']

0.089999999999999997